# installation of detectron2, streamlit and yolo

In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 2.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
torch:  2.4 ; cuda:  cu121
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-_l0y4rrj
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-_l0y4rrj
  Resolved https://github.com/facebookresearch/detectron2.git to commit 5b72c27ae39f99db75d43f18fd1312e1ea934e60
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install streamlit -q # installing streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
!pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.0/872.0 kB 15.0 MB/s eta 0:00:00


# writing streamlit app

In [ ]:
%%writefile app.py
import streamlit as st
import os
import cv2
import numpy as np
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo
import torch
from PIL import Image
from ultralytics import YOLO
import io

# Load class names from a file
def load_class_names(filepath):
    with open(filepath, 'r') as file:
        class_names = [line.strip() for line in file]
    return class_names

# Load Detectron2 configuration
def setup_detectron2():
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/retinanet_R_101_FPN_3x.yaml'))
    cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/detectron2/output/model_0000999.pth'  # Final model weights
    cfg.MODEL.DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    return DefaultPredictor(cfg)

# Predict and draw boxes with Detectron2
def predict_and_draw_boxes_detectron2(image, predictor, class_names, threshold=0.4):
    # Convert image from RGB to BGR for OpenCV
    img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    outputs = predictor(img)
    preds = outputs["instances"].pred_classes.tolist()
    scores = outputs["instances"].scores.tolist()
    bboxes = outputs["instances"].pred_boxes.tensor.cpu().numpy()

    for j in range(len(scores)):
        bbox = bboxes[j]
        score = scores[j]
        pred = preds[j]
        if score > threshold:
            x1, y1, x2, y2 = [int(i) for i in bbox]
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
            label = f"{class_names[pred]}: {score:.2f}"
            font_scale = 1.2
            font_thickness = 2
            text_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
            text_w, text_h = text_size
            cv2.rectangle(img, (x1, y1 - text_h - 5), (x1 + text_w, y1), (255, 0, 0), -1)
            cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)

    # Convert image back from BGR to RGB for Streamlit display
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Predict and draw boxes with YOLO
def predict_and_draw_boxes_yolo(image, model, threshold=0.3):
    # Convert image from PIL to OpenCV format (RGB to BGR)
    img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    results = model(img)

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        confidences = result.boxes.conf.cpu().numpy()
        class_ids = result.boxes.cls.cpu().numpy()
        class_names = result.names

        for (box, confidence, class_id) in zip(boxes, confidences, class_ids):
            if confidence < threshold:
                continue
            x1, y1, x2, y2 = map(int, box)
            height, width = img.shape[:2]
            x1, y1, x2, y2 = max(0, x1), max(0, y1), min(width, x2), min(height, y2)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 10)
            label = f'{class_names[int(class_id)]}: {confidence:.2f}'
            font_scale = 1
            font_thickness = 4
            label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)[0]
            label_y = max(y1 - label_size[1], 0)
            cv2.putText(img, label, (x1, label_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)

    # Convert image back from BGR to RGB for Streamlit display
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Streamlit app setup
st.title("Biodiversity Net Gain")
st.header("Please Upload an Image")

# Model selection
model_option = st.selectbox("Select Model", ("Detectron2", "YOLO"))

# Upload the image
uploaded_file = st.file_uploader('', type=['png', 'jpg', 'jpeg'])

# Load class names
class_names_file = '/content/drive/MyDrive/detectron2/class.names'
class_names = load_class_names(class_names_file)

# Set up the models based on the selection
if model_option == "Detectron2":
    predictor = setup_detectron2()
elif model_option == "YOLO":
    model_path = "/content/drive/MyDrive/Dissertation Dataset/models/runs/detect/train2/weights/best.pt"
    model = YOLO(model_path)

if uploaded_file is not None:
    # Convert the uploaded file to an image using BytesIO
    image = Image.open(io.BytesIO(uploaded_file.read()))

    if model_option == "Detectron2":
        result_img = predict_and_draw_boxes_detectron2(image, predictor, class_names)
    elif model_option == "YOLO":
        result_img = predict_and_draw_boxes_yolo(image, model)

    # Display the image with bounding boxes
    st.image(result_img, caption='Predicted Image', use_column_width=True)



Writing app.py


# running streamlit app

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.90.168:8501

Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) y
your url is: https://nasty-bikes-enter.loca.lt
2024-09-02 16:53:55.606 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
Loading config /usr/local/lib/python3.10/dist-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
2024-09-02 16:55:00.326 `label` got an empty value. This is discouraged for accessibility reasons and may be disallowed in the future by raising an exception. Please provide a non-empty label and hide it with label_visibility if needed.
Loading co